**Compare two pandas dataframes for equality**

Specify df1, df2, and absolute/relative tolerance for numeric comparison.

1. Check Shapes - Number of (rows, cols) - Match

2. Check Column Names Match

3. Check Values Match

  * Values are compared by index - indices that exist in both frames are compared
  * Numeric values are compared with `numpy.isclose`
  * Object values are compared with `numpy.equal`

In [1]:
import pprint
import pandas as pd
import numpy as np

results = dict(setup=[], shapes=[], columns=[], values=[])
def print_results(r):
    for k, v in r:
        print(k + ':', v)

In [2]:
# specify input settings
fname1 = 'data1.csv'
fname2 = 'data2.csv'
desc = 'Comparison of df1 and df2'

df1 = pd.read_csv(fname1)
df2 = pd.read_csv(fname2)

atol = 1e-4
rtol = 0

In [3]:
r = results['setup']
r.clear()

r.append(('Filename 1', fname1))
r.append(('Filename 2', fname2))
r.append(('Description', desc))
r.append(('Absolute Numeric Tolerance', atol))
r.append(('Relative Numeric Tolerance', rtol))

print_results(r)

Filename 1: data1.csv
Filename 2: data2.csv
Description: Comparison of df1 and df2
Absolute Numeric Tolerance: 0.0001
Relative Numeric Tolerance: 0


In [4]:
# Compare Shapes (row, col)
r = results['shapes']
r.clear()

r.append(('Shapes Match', df1.shape == df2.shape))
for i, df in enumerate([df1, df2]):
    r.append(('df{} Shape'.format(i+1), df.shape))
    
print_results(r)

Shapes Match: False
df1 Shape: (7, 4)
df2 Shape: (6, 3)


In [5]:
# Compare Columns
r = results['columns']
r.clear()

r.append(('Columns Match', df1.shape == df2.shape))
for i, df in enumerate([df1, df2]):
    r.append(('df{} Columns'.format(i+1), df.columns.values))
cols = df1.columns.difference(df2.columns).values
r.append(('df1 Only', cols))
cols = df2.columns.difference(df1.columns).values
r.append(('df2 Only', cols))
print_results(r)

Columns Match: False
df1 Columns: ['char' 'num' 'mix' 'extra']
df2 Columns: ['char' 'num' 'mix']
df1 Only: ['extra']
df2 Only: []


In [6]:
# Compare values
r = results['values']
r.clear()

x = df1.values
y = df2.values

# min / max size of arrays
min_dim = tuple(min(a, b) for a, b in zip(x.shape, y.shape))
max_dim = tuple(max(a, b) for a, b in zip(x.shape, y.shape))

# numeric columns
x_num = [np.issubdtype(t, np.number) for t in df1.dtypes]
y_num = [np.issubdtype(t, np.number) for t in df2.dtypes]
cols_num = np.logical_and(x_num[:min_dim[1]], y_num[:min_dim[1]])

# numeric check
x = df1.iloc[:min_dim[0], cols_num].values
y = df2.iloc[:min_dim[0], cols_num].values
elements = np.where(~np.isclose(x, y, atol=atol, equal_nan=True))
col_map = np.arange(len(cols_num))[cols_num]
rows = elements[0]
cols = np.array([col_map[col] for col in elements[1]])
idx = np.column_stack((rows, cols))

# obj check
x = df1.iloc[:min_dim[0], ~cols_num].values
y = df2.iloc[:min_dim[0], ~cols_num].values
elements = np.where(x != y)
col_map = np.arange(len(cols_num))[~cols_num]
rows = elements[0]
cols = np.array([col_map[col] for col in elements[1]])
idx = np.concatenate((idx, np.column_stack((rows, cols))))

# table of mismatch values
columns = ("i", "j", "col1", "col2", "val1", "val2")
data = []
for i, j in idx:
    z = ((i, j, df1.columns[j], df2.columns[j], df1.iloc[i, j], df2.iloc[i, j]))
    data.append(z)
df = pd.DataFrame(data, None, columns)

r.append(('Values Match', df.empty))
r.append(('Max Shape', max_dim))
r.append(('Possible Values', np.prod(max_dim)))
r.append(('Excluded Values', np.prod(max_dim) - np.prod(min_dim)))
r.append(('Shape Compared', min_dim))
r.append(('Total Values Compared', np.prod(min_dim)))
r.append(('Total Values Matched', np.prod(min_dim) - idx.shape[0]))
r.append(('Total Values Not Matched', df.shape[0]))

print_results(r)
df

Values Match: False
Max Shape: (7, 4)
Possible Values: 28
Excluded Values: 10
Shape Compared: (6, 3)
Total Values Compared: 18
Total Values Matched: 15
Total Values Not Matched: 3


,i,j,col1,col2,val1,val2
0,1,1,num,num,2,NaN
1,4,1,num,num,5,2
2,2,0,char,char,c,a


In [7]:
# export results
with open('report.txt', 'w') as f:
    f.write(pprint.pformat(results))
df.to_csv('results_data.csv', index=False)